# TIE Reconstruction Template
For solving the transport of intensity equation and finding magnetic induction from through-focal-series TEM images. 
  
Authors: Arthur McCray, CD Phatak  
V1.0, ANL, May 2020

## Refer to readme file for instructions

Required files:

    TIE_reconstruct.py  -- main code that sets up and solves TIE  
    TIE_params.py       -- class file for reconstruction parameters  
    microscopes.py      -- class file for microscope parameters
    TIE_helper.py       -- functions for loading data, making masks, etc.   
    colorwheel.py       -- makes the colorwheel vectorfield representation image  
  
    stack_align.ipynb   -- interfaces with imagej to align images, not required 
                           but very helpful
  

### Future development
* Apply polynomial fitting for the derivative

In [ ]:
%matplotlib notebook
import numpy as np
import matplotlib.pyplot as plt

from microscopes import Microscope
from TIE_helper import *
from TIE_reconstruct import TIE, SITIE
# will throw warning from hyperspy. You can also use the Qt5 backend with:
# %matplotlib qt
# This will get rid of the warning but won't work when running remotely. 

## Loading the data

Please download the example dataset from: https://petreldata.net/mdf/detail/cphatak_pytie_example_v1.1/  
This contains a full through focus series (tfs) with the sample in both flipped and unflipped orientations. 

If you have both a flip and unflip stack your raw images should be in two folders:  

    datafolder/    flip/     -im1.dm3  
                             -im2.dm3  
                                ...  
                             +im1.dm3  
                             +im2.dm3  
                                ...  
                             0im.dm3    
                   unflip/   -im1.dm3  
                             -im2.dm3  
                                 .  
                                 .  
                             +im1.dm3  
                             +im2.dm3  
                                 .  
                                 .  
                              0im.dm3  
                   flsfile.fls 
                   full_align.tif  
  
If your flip and unflip filenames aren't the same you can also have two fls files, just add the argument  
load_data(... , flip_fls_file = "your/flip/fls/path")

However if you have just one stack (no flip stack) then your data should be in a folder labeled 'tfs' 

    datafolder/    tfs/      -im1.dm3  
                             -im2.dm3  
                                ...  
                             +im1.dm3  
                             +im2.dm3  
                                ...  
                              0im.dm3    
                   flsfile.fls 
                   full_align.tif  

In [ ]:
# fill in the appropriate path *** 
data_loc = '../../ex_recon_data/PyTIE_MDF_data/'

fls_name = "dataset1.fls"
flip_fls_name = None # just one fls file
stack_name = 'd1_full_align.tif' 
flip = True # there is/isn't a flipped tfs and flip_dm3stack

dm3stack, flip_dm3stack, ptie = load_data(data_loc,
                                          fls_name, 
                                          stack_name, 
                                          flip,
                                          flip_fls_name)

print("Defocus values: ", ptie.defvals)

# microscope object
altem = Microscope(E=200e3, Cs = 200.0e3, theta_c = 0.01e-3, def_spr = 80.0)

In [ ]:
# Scroll through slices 
show_stack(dm3stack + flip_dm3stack)

## Select a smaller image size (optional)
* Default is 1/4 image size centered in the middle
* Drag mask and corners to get the section you want to reconstruct
    - It's finnicky, if the corner isn't dragging, try moving mouse cursor more slowly and make sure you're on the corner. 
    - if it still isn't working you can change ptie.roi by hand (it's in nm not pixels, convert with ptie.scale which is nm/pix)

In [ ]:
ptie.select_region()

In [ ]:
### Run this when the region is what you want. 
ptie.crop_dm3s()

In [ ]:
show_stack(ptie.dm3stack+ptie.flip_dm3stack, ptie)

## The actual phase reconstruction

* Set which focus values you want to use with index i 
* images will be saved as /dataloc/images/dataname_focusval_imagename.tif
* if you want to symmetrize the images set sym = True
* if you want to run with a Tikhonov frequency
    - set qc = value that you want, (currently needs large values ~>10) or 
    - set qc = 'percent' and it will use 15% of q (not that it makes much difference) or
    - leave qc = None and it will run normally
* Save options:  
    - True    -- saves all images results directory
    - 'b'     -- saves 'color_b', 'bxt', 'byt'
    - 'color' -- saves just 'color_b'
    - False   -- Does not save images, but they can still be viewed in the notebook
    - If any images are saved, a dataname_params.txt file will be saved as well giving reconstruction parameters. 

In [ ]:
print('actual defocus values: ', ptie.defvals)
print('reconstruct with   i = ', '    0    ', '   1   ', '     2 ...')

In [ ]:
i = 2 # Select which defocus value you'll be working with if 3-point deriv 
dataname = 'example' 
sym = False
qc = False # 0.001 is a reasonable value for the test dataset, matters more on some data
save = False # True for saving all of results, 'b' will save 'color_b', 'bxt', and 'byt', 'color' will just save 'color_b'
longitudinal_deriv = False # If false, uses central difference (3 point)
hsv = True # If false, this will create a 4-fold colorwheel instead of an hsv colorwheel
print(f'Set to reconstruct for defocus value: {ptie.defvals[i]} nm ')

In [ ]:
results = TIE(i, ptie, altem, 
                     dataname = dataname, 
                     sym=sym, 
                     qc = qc, 
                     save=save,
                     hsv=hsv,
                     long_deriv = longitudinal_deriv)

## Viewing the images
You can of course just look at the data where it's saved, but if you don't want to save it and move it to a local computer you can view the images here, they're all in the results dictionary. 
* Results: 
    - 'byt' : y-component of integrated magnetic induction,
    - 'bxt' : x-copmonent of integrated magnetic induction,
    - 'bbt' : magnitude of integrated magnetic induction, 
    - 'phase_m' : magnetic phase shift (radians),
    - 'phase_e' : electrostatic phase shift (if using flip stack) (radians),
    - 'dIdZ_m' : intensity derivative for calculating phase_m, (useful for troubleshooting)
    - 'dIdZ_e' : intensity derivative for calculating phase_e (if using flip stack), 
    - 'color_b' : RGB image of magnetization,
    - 'inf_im' : the in-focus image

In [ ]:
show_im(results['phase_m'], 'magnetic phase shift')

In [ ]:
show_im(results['phase_e'], 'electrostatic phase shift')

In [ ]:
## Show the image with a four-fold colorwheel instead of a 3-fold hsv colormap
from colorwheel import color_im
show_im(color_im(results['bxt'], results['byt'], hsvwheel=False), "Magnetization with four-fold colormap")

### Single Image Reconstruction (SITIE)
Uniformly thin magnetic samples can be reconstructed from a single defocused image if their only source of contrast is magnetic Fresnel contrast. For an example of this please see /SimLTEM/Sim_example.ipynb

--- end notebook ---